In [7]:
import keras
import os

In [10]:
weights = os.path.join("C:\\","Users", "robin", "Desktop", "Unibas", "PROJECT", "model", "CNN_MODEL", "weights", "finetune", "fine_tuned_0.h5")

In [4]:
categories = ["APIACEAE", "ARTEMISIA", "ARTEMISIA_SIDE", "BETULA", "BETULA_SIDE","CHENOPODIACEAE", 
              "CYPERACEAE", "HELIANTHEMUM","LYCOPODIUM", "LYCOPODIUM_HALF", "NPP_CLASS1", "NPP_CLASS2", 
              "NPP_CLASS3", "NPP_CLASS4", "PINUS_WING", "POACEAE", "POACEAE_SIDE", "SALIX"]

In [5]:
cnnInputSize = (140,140) # Input Shape for classification module.
img_width, img_height = cnnInputSize

def create_model():

    feature_extractor = keras.applications.xception.Xception(
        weights="imagenet",  # Load weights pre-trained on ImageNet.
        include_top=False,
        input_shape = (img_width,img_height,3),
    ) 


    feature_extractor.trainable = False

    input = feature_extractor.output
    x = keras.layers.GlobalAveragePooling2D()(input)
    x = keras.layers.Dense(200)(x)
    x = keras.layers.Dropout(0.25)(x)
    x = keras.layers.Dense(len(categories))(x)
    output = keras.layers.Softmax()(x)


    model = keras.models.Model(inputs = feature_extractor.input, outputs = output)


    model.summary()

    return model

In [6]:
model = create_model()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 140, 140, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 69, 69, 32)           864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 69, 69, 32)           128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 69, 69, 32)           0         ['block1_conv1_bn[0][0]'

In [12]:
model.load_weights(weights)

In [13]:
!mkdir -p saved_model

In [14]:
model.save("saved_model/FossilPollenModel")

INFO:tensorflow:Assets written to: saved_model/FossilPollenModel\assets


INFO:tensorflow:Assets written to: saved_model/FossilPollenModel\assets


In [4]:
import tensorflow as tf

In [5]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model/FossilPollenModel")
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)